In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
import tensorflow as tf
from keras.models import Model, Sequential
from keras.models import load_model, save_model
import glob 
import  pandas as pd

from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
import sklearn as sk

from datetime import datetime
from datetime import timedelta
from keras.layers import Normalization
from keras.layers import InputLayer,Subtract, Dot, Reshape, Input, Permute
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
version= 'v0.0.13'
workspace= "G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\workspace\\data\\training"
assests_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Zollern-FH-MillTrunMotor\\Datasets\\workspace\\assets\\New_Motor\\'+version+'\\'
preprocessor_dir=assests_dir
preprocessor_name='preprocessor.p'
model_dir= assests_dir
model_name= 'model.h5'

In [ ]:
### load models
pipeline= load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model= load_model(str(model_dir + model_name), compile= False)
print('Model is loaded successfully')
print (model.summary())

Save Model as .keras model

In [ ]:
model.save(str(model_dir + 'model.keras'))

In [ ]:
pipeline.transform([[1,2,3,4]])

In [ ]:
pca_comp= pipeline.steps[1][1].components_.reshape((1,4,4))

In [ ]:
pca_comp.reshape(1,-1)

Covert the pipeline to a keras code

Using constant to pass to model

In [ ]:
## Not Working and it should be checked
# input2_constant_ = tf.constant(pca_comp)
# input1= Input(shape=(4,), name= "Input_layer")
# #input2= Input(shape=(4,4), name="cov_matrix", tensor= input2_constant)
# normalized_data = Normalization(mean=pipeline.steps[0][1].mean_, variance=pipeline.steps[0][1].scale_ ** 2, name= "Normalization_layer1")(input1)
# reshaped_normalized_data = normalized_data#Reshape((4, 1))(normalized_data)
# input2_constant= input2_constant_#Permute((2, 1))(input2_constant_)
# #reshaped_transposed_components = Reshape((4, 4))(input2_constant)
# matrix_product = Dot(axes=(1,2) , name= "Dot_Layer")((reshaped_normalized_data, input2_constant))
# normalized_pca= Normalization(name= "Normalization_layer2", mean= pipeline.steps[2][1].mean_, variance= pipeline.steps[2][1].scale_** 2)(matrix_product)
# keras_preprocessor = Model(inputs=input1, outputs=normalized_pca, name= "Preprocessor",)
# keras_preprocessor.compile()
# test_input= np.asarray([[5,8,4,9]])
# output_test= keras_preprocessor.evaluate(x= test_input)
# print (keras_preprocessor.predict(test_input))
# print(output_test)
# print ("Model Input: ",keras_preprocessor.input_shape, "Model Output: ", keras_preprocessor.output_shape)


In [ ]:
# keras_preprocessor.save(str(model_dir + 'preprocessor2.keras'))
# loaded_model= load_model(str(model_dir + 'preprocessor2.keras'))
# test_input= np.asarray([[5,8,4,9]])
# loaded_model.predict([test_input, pca_comp])

Model with cov-Matrix as second Input

In [ ]:
input1= Input(shape=(4,), name= "Input_layer")
input2= Input(shape=(4,4), name="cov_matrix")
normalized_data = Normalization(mean=pipeline.steps[0][1].mean_, variance=pipeline.steps[0][1].scale_ ** 2, name= "Normalization_layer1")(input1)
#reshaped_transposed_components = Permute((2,1))(input2)
matrix_product = Dot(axes= (1,2), name= "Dot_Layer")((normalized_data, input2))
normalized_pca= Normalization(name= "Normalization_layer2", mean= pipeline.steps[2][1].mean_, variance= pipeline.steps[2][1].scale_** 2)(matrix_product)
keras_preprocessor = Model(inputs=[input1, input2],outputs=normalized_pca, name= "Preprocessor")
keras_preprocessor.compile()
test_input= np.asarray([[5,8,4,9]])
output_test= keras_preprocessor.evaluate(x= [test_input,pca_comp])
print (keras_preprocessor.predict([test_input,pca_comp]))
print(output_test)
print ("Model Input: ",keras_preprocessor.input_shape, "Model Output: ", keras_preprocessor.output_shape)

In [ ]:
pipeline.transform(test_input)

In [ ]:
keras_preprocessor.save(str(model_dir + 'preprocessor.keras'))
loaded_model= load_model(str(model_dir + 'preprocessor.keras'))
test_input= np.asarray([[5,8,4,9]])
loaded_model.predict([test_input, pca_comp])

In [ ]:
pipeline.transform(test_input)